# 머신러닝 초보를 위한 MNIST
(https://codeonweb.com/entry/12045839-0aa9-4bad-8c7e-336b89401e10)


이 연습은 기계 학습 및 TensorFlow 양쪽 모두 처음인 사람들을 위한 것입니다.  
만약 MNIST가 무엇인지 이미 알고 있고, 소프트맥스 회귀 (softmax (multinomial logistic) regression) 가 무엇인지 알고 있다면, 훨씬 빠른 연습을 보는 것을 권합니다. 
(https://codeonweb.com/entry/f50e23df-0f23-4e56-95a6-efb9981716f7)


사람이 어떻게 프로그래밍하는지 배울 때, 가장 처음 하는 것은 관례적으로 "Hello World" 를 출력해 보는 것입니다.  
프로그래밍에서 Hello World가 있듯이, 기계 학습에는 MNIST가 있습니다.


MNIST 는 간단한 컴퓨터 비전 데이터 세트입니다.  
아래와 같이 손으로 쓰여진 이미지들로 구성되어 있습니다.

https://resources.codeonweb.com/bucket/cached/b8/d2/b8d2af84ecc722a5ac671a2b4697d887.png

이 데이터는 각 데이터의 숫자가 무엇인가에 대한 라벨이 함께 붙어 있습니다.  
예를 들어, 해당 이미지들에는 5, 0, 4, 1 이라는 라벨들이 붙어 있습니다.  

이 연습에서, 우리는 이미지들을 들여다보고 그 이미지들이 어떤 숫자인지 예측하는 학습 모델을 만들겁니다.  
우리 목표는 아주 예술적인 성능이 나오는 엄청 정교한 모델을 학습하는 것 (비록 뒤에서 그걸 할 수 있는 코드도 주겠지만) 이 아니라, TensorFlow에 발끝만 살짝 담그는 겁니다.   
그러니, 우리는 아주 간단한 모델인 소프트맥스 회귀(Softmax Regression) 부터 시작하겠습니다.  

이 예시의 실제 코드들은 엄청나게 짧습니다.   
그리고, 모든 재미있는 것들은 딱 세 줄 안에서 일어납니다.   
하지만, 그 뒤에 어떤 아이디어들이 있는지 (TensorFlow가 어떻게 동작하는지라거나, 기계학습의 핵심 개념 등을)이해하는 것은 아주 중요합니다.   
그렇기 때문에 우리는 코드들을 아주 주의깊게 들여다 보겠습니다.

---

### MNIST 데이터

MNIST 데이터는 Yann LeCun의 웹사이트에서 제공합니다.  
편향을 위해 우리가 데이터를 자동으로 다운로드하고 설치하는 코드를 포함해 놓았습니다.  
코드를 다운로드 하고 아래와 같이 import하거나, 그냥 안에 붙여 넣으시면 됩니다.

(source code: https://github.com/tensorflow/tensorflow/blob/r0.7/tensorflow/examples/tutorials/mnist/input_data.py)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Dataset loading
mnist = input_data.read_data_sets("./samples/MNIST_data/", one_hot=True)

Extracting ./samples/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./samples/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/t10k-labels-idx1-ubyte.gz


다운로드한 데이터는 55000개의 학습 데이터 (mnist.train), 10000개의 테스트 데이터 (mnist.test) 및 5000개의 검증 데이터 (mnist.validation) 세 부분으로 나누어져 있습니다.  
이렇게 나눠진 것은 굉장히 중요합니다.  
학습하지 않은 **독립된 데이터**로 학습한 결과가 실제로 일반적인지 검증하는 것이 기계 학습의 핵심입니다!

앞에서 이야기했듯, 모든 MNIST 데이터 포인트들은 두 부분으로 되어 있습니다.  
**손으로 쓴 숫자**와 그에 해당하는 **라벨**입니다.  
우리는 **이미지를 "xs"**라고 부르고 **라벨을 "ys"**라고 부르겠습니다.  
학습 데이터 세트와 테스트 데이터 세트는 xs와 ys를 포함하고 있습니다.   
예를 들어, 학습 세트의 이미지들은 mnist.train.images 이며, 학습 세트의 라벨은 mnist.train.labels 입니다.

각 이미지는 가로세로 28픽셀씩입니다.  
우리는 이걸 숫자들로 구성된 큰 행렬로 취급할 수 있습니다.  

https://resources.codeonweb.com/bucket/cached/d4/e5/d4e5709ebb4ba940126de44c76ca71b0.png

우리는 이 행렬을 28x28 = 784개의 숫자를 갖는 벡터로 단조화할 수 있습니다.  
우리가 이미지들에 모두 동일한 방법을 사용하는 한, 어떻게 행렬을 단조화하는지는 중요하지 않습니다.  
이 관점에서 보면 MNIST 이미지들은 그저 784 차원 벡터 공간 안의 아주 풍부한 구조를 지닌 점들의 모음일 뿐입니다.  
(주의: 컴퓨터 자원이 많이 필요한 시각화들입니다)


데이터를 단조화하는 과정은 이미지의 2차원 구조를 버립니다.  
이게 나쁜가요? 글쎄요, 최고의 컴퓨터 비전 방법들은 이 구조도 전부 이용하고, 우리도 이후의 예제에서 해 보겠습니다.  
그러나 우리가 여기서 이용할 간단한 방법인 소프트맥스 회귀에서는 사용하지 않습니다.


결과로 mnist.train.images 는 [55000,784] 형태의 텐서 (n 차원 행렬) 를 얻었습니다.  
첫번째 차원 인덱스는 이미지들을, 두번째 차원 인덱스들은 각 이미지들의 픽셀에 대응됩니다.  
텐서의 각 구성 요소들은 특정한 이미지 안의 특정한 픽셀의 진하기를 0에서 1 사이의 값으로 나타냅니다.

https://resources.codeonweb.com/bucket/cached/01/44/01442103dfbc7159abd9382d832fb07e.png

MNIST 안에서 대응되는 라벨들은 주어진 각 이미지가 어떤 숫자를 나타내는가를 의미하는 0에서 9사이의 숫자입니다.  
이 연습의 목적으로, 우리는 우리 라벨들을 "one-hot 벡터" 로 사용할겁니다.  
one-hot 벡터는 하나의 차원만 1이고 나머지 모든 차원들은 0으로 채워진 벡터입니다.  
이 경우, nn번째 숫자는 nn번째 차원이 1인 벡터로 표시될 것입니다.  
예를 들어, 3은 [0,0,0,1,0,0,0,0,0,0] 로 표현됩니다.  
따라서 mnist.train.labels 는 [55000, 10] 행렬이 됩니다.  

https://resources.codeonweb.com/bucket/cached/29/32/2932868078c91f45f92b2eefa005ff4a.png


이제 모델을 만들 준비가 다 됐습니다!

---


### Softmax Regression

우리는 MNIST 의 모든 이미지가 0부터 9까지의 숫자인 것을 알고 있습니다.  
우리는 이미지들을 들여다보고 각 숫자들에 확률을 주려고 합니다.  
예를 들어, 우리 모델은 9의 사진을 보고 그 숫자가 80% 정도 9라고 확신하지만, 동시에 (숫자 윗쪽의 곡선 영향으로) 8일 가능성도 5%를 주고, 나머지 모든 숫자들에는 확신할 수 없으므로 아주 작은 확률을 줄 것입니다. 


이 경우는 소프트맥스 회귀가 자연스럽고 간단한 모델인 고전적인 경우입니다.  
만약 어떤 대상에 여러 다양한 것들 중 하나일 확률을 매기려면 소프트맥스가 가장 적당합니다.  
심지어 나중에 우리가 훨씬 더 정교한 모델들을 배웠을 때에도, 마지막 단계는 소프트맥스 레이어일 것입니다.


소프트맥스 회귀는 두 단계가 있습니다:  
첫번째로 우리 입력이 특정 클래스에 해당되는지에 대한 증거를 더하고, 그 다음 증거를 확률로 변환합니다.


주어진 이미지가 특정한 클래스에 들어가는지의 증거를 모아 계산하기 위해 픽셀 농도의 가중합을 사용합니다.  
특정 클래스 안의 이미지들의 픽셀들에 비해 픽셀 농도가 높을 경우 가중치는 음수이고, 그렇지 않을 경우 가중치는 양수가 됩니다.

아래의 다이어그램은 세가지 클래스 각각에 대해 학습한 모델의 가중치를 보여줍니다.  
빨간색은 음수 가중치를, 파란색은 양수 가중치를 나타냅니다. 

https://resources.codeonweb.com/bucket/cached/9b/79/9b792345c0394ce51d9b08b66287f5fd.png


이제 편향(bias)이라고 부르는 추가적인 증거를 더합니다.  
기본적으로, 우리는 몇몇 경우들은 입력들에 대해 더 자유롭다고 말할 수 있게 하고 싶죠.  
결과적으로 주어진 입력 x 에 대한 클래스 i 의 증거는


evidencei=∑jWi, jxj+bievidencei=∑jWi, jxj+bi


이 됩니다.  
Wi 는 가중치, bi 는 클래스 i 에 대한 편향, j 는 입력한 이미지 x 의 픽셀들에 따른 합을 구하기 위한 인덱스입니다.  
이제 증거 항목들을 "소프트맥스" 함수를 이용해 예측 확률로 변환합니다:

y=softmax(evidence)y=softmax(evidence)

여기서 소프트맥스가 우리가 제공한 선형 함수의 출력 결과를 원하는 형태로 만들기 위해 "활성화" 나 "링크" 함수의 형태로 적용됩니다 -- 이 경우, 10가지 경우에 대한 확률 분포입니다.  
이것을 증거 항목들을 각 클래스에 대응하는 확률로 변환하는 과정으로 생각해도 됩니다.  
이 과정은 다음과 같이 정의합니다:

softmax(x)=normalize(exp(x))softmax(x)=normalize(exp⁡(x))

이 식을 전개하면 다음과 같은 결과를 얻습니다:

softmax(x)i=exp(xi)∑jexp(xj)softmax(x)i=exp⁡(xi)∑jexp⁡(xj)

그러나 종종 소프트맥스를 입력을 지수화하고 정규화하는 첫번째 방법으로 생각하는 것이 훨씬 도움이 됩니다.  
지수화는 증거에 하나가 더해질 경우 가중치를 곱으로 증가시키는 의미가 됩니다.  
반대로 말하면, 증거에서 하나가 줄어들면 가설의 가중치가 원래 가중치의 분수비로 줄어든다는 의미가 됩니다. 
어떤 가설도 0이나 음수의 가중치를 가질 수 없습니다.  
그런 후 소프트맥스는 이러한 가중치를 정규화해서, 모두 더하면 1이 되는 확률 분포의 형태로 만듭니다.  
(소프트맥스 함수에 대한 더 많은 직관을 얻고 싶다면, 시각화가 있는 마이클 닐센의 책의 장을 참고하세요.)

소프트맥스 회귀를 아래와 같이 그려볼 수 있습니다 (훨씬 많은 x가 있다는 것만 제외하면요) 각각의 출력마다, 우리는 가중치의 합을 계산하고, 편향을 더하고, 소프트맥스를 적용합니다.

https://resources.codeonweb.com/bucket/cached/3d/eb/3debe13595e70edb78dfe62fe562688a.png


식 형태로 쓰면:

https://resources.codeonweb.com/bucket/cached/c4/fe/c4febfa77b4a0c19e81e41eb6d989800.png

우리는 이 과정을 행렬곱과 벡터합으로 바꾸는 식으로 "벡터화" 할 수 있습니다. 벡터화는 계산 효율을 위해서 도움이 됩니다. (또한 생각하기에도 유용합니다)

https://resources.codeonweb.com/bucket/cached/fa/99/fa991ee0070b1abad576b2edc0aaa54f.png

더 간단하게는 그냥 이렇게 쓸 수 있습니다:

y=softmax(Wx+b)

---

### Regression 구현하기

Python 으로 효율적인 수치 연산을 하기 위해서 우리는 보통 행렬곱 등의 무거운 연산을 Python 외부에서 처리하는 NumPy등의 라이브러리들을 사용하는데, 이들은 다른 언어로 구현한 고도로 최적화된 코드를 이용합니다.  
불행히도, 모든 작업 단계에서 Python으로 도로 스위칭하는 과정에서도 아주 큰 오버헤드가 여전히 존재할 수 있습니다.  
이러한 오버헤드는 GPU를 이용하거나 분산 처리를 이용하는, 데이터 전송에 큰 비용을 지불하는 환경에서 계산을 실행할 경우 특히 큽니다.

TensorFlow 또한 Python 외부에서 무거운 작업들을 실행하지만, 이러한 오버헤드를 피하기 위해 한 단계 더 나아갔습니의 일을 더 수행합니다. 
Python과 독립된 큰 단일 작업을 실행하는 대신, TensorFlow는 완전히 Python 밖에서 실행되는 서로 상호작용하는 작업들의 그래프를 기술하도록 합니다. (몇몇 기계학습 라이브러리들에서 이러한 접근법을 볼 수 있습니다)

심볼릭 변수들을 사용하여 상호작용하는 작업들을 기술합니다. 하나 만들어 봅시다:

In [3]:
# Set up model
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

x 는 특별한 값이 아닙니다.  
수식기호인데, 우리가 TensorFlow에게 계산을 하도록 명령할 때 입력할 값입니다.  
우리는 MNIST 이미지들의 어떤 숫자들이든 입력할 수 있기를 원하는데, 각 이미지들은 784차원의 벡터로 단조화되어 있습니다. 
우리는 이걸 [None, 784] 형태의 부정소숫점으로 이루어진 2차원 텐서로 표현합니다. (None 은 해당 값이 어떤 길이도 될 수 있음을 의미합니다.)

우리는 또한 가중치와 편향값이 필요합니다.  
이런 것들을 추가적인 입력으로 다루는 것을 생각해 볼 수 있습니다.  
그러나 TensorFlow는 더 나은 방법을 갖고 있습니다: Variable이죠.  
Variable은 TensorFlow의 상호작용하는 작업 그래프들간에 유지되는 변경 가능한 텐서입니다.  
계산 과정에서 사용되거나 심지어 변경될 수도 있습니다.  
기계학습 응용 사례들을 위해 일반적으로 모델 파라미터 Variables 를 사용합니다.

우리는 tf.Variable 을 주어서 Variable의 초기값을 만듭니다: 이 경우, 우리는 W 와 b 를 0으로 채워진 텐서들로 초기화합니다.  
우리가 W와 b 를 학습할 것이기 때문에, 그것들이 무엇으로 초기화되었는지는 크게 중요하지 않습니다.

W의 형태가 [784, 10] 임을 주의합시다.  
우리는 784차원의 이미지 벡터를 곱하여 10차원 벡터의 증거를 만들것이기 때문입니다.  
b는 [10]의 형태이므로 출력에 더할 수 있습니다.

---

이제 우리 모델을 구현할 수 있습니다. 딱 한 줄이면 됩니다!

첫번째로, 우리는 tf.matmul(x, W) 표현식으로 x 와 W를 곱합니다.  
이 값은 Wx가 있던 우리 식에서 곱한 결과에서 뒤집혀 있는데, 이것은 xx가 여러 입력으로 구성된 2D 텐서일 경우를 다룰 수 있게 하기 위한 잔재주입니다.  
그 다음 b를 더하고, 마지막으로 tf.nn.softmax 를 적용합니다.

됐습니다.  
몇몇 짧은 설정 줄들에 이어 단지 한 줄로 우리 모델을 정의할 수 있었습니다.  
이것은 TensorFlow 가 소프트맥스 회귀를 특히 쉽게 할 수 있게 디자인되어 있어서가 아닙니다: 그저 기계학습 모델부터 물리학 시뮬레이션까지 다양한 종류의 수치 계산을 다룰 수 있는 아주 유연한 방법입니다.  
그리고 우선 한 번 정의하면, 우리 모델은 다양한 기기에서 실행할 수 있습니다: 컴퓨터의 CPU, GPU들, 심지어는 핸드폰에서도요!

In [10]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [11]:
y_ = tf.placeholder(tf.float32, [None, 10])

In [12]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [13]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [14]:
# Session
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

In [15]:
# Learning
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [16]:
# Validation
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Showing result
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.92
